# Symbolic Calculation of Full Jacobian

In [ ]:
from sympy import *
import numpy as np
init_printing(use_unicode=True)

In [2]:
def dh_transform(a, alpha, d, theta):
    return Matrix([
        [cos(theta), -sin(theta)*cos(alpha), sin(theta)*sin(alpha), a*cos(theta)],
        [sin(theta), cos(theta)*cos(alpha), -cos(theta)*sin(alpha), a*sin(theta)],
        [0, sin(alpha), cos(alpha), d],
        [0, 0, 0, 1]
    ])

def rotation_to_euler(R):
    yaw = atan2(R[1, 0], R[0, 0])
    pitch = atan2(-R[2, 0], sqrt(R[2, 1]**2 + R[2, 2]**2))
    roll = atan2(R[2, 1], R[2, 2])
    return Matrix([yaw, pitch, roll])

In [11]:

# Define symbolic joint angles
theta = symbols('theta1:7')

l1, l2, h1, h2, w, h3, h4, o = symbols('l1, l2, h1, h2, w, h3, h4, o')

dh_params = [
    (0, pi/2, h1, theta[0]),
    (l1, 0, 0, theta[1] + pi/2),
    (h2, pi/2, 0, theta[2] + pi/2),
    (0, -pi/2, l2 + h3, theta[3]),
    (0, pi/2, h4, theta[4]),
    (0, 0, o, theta[5])
]

# Compute forward kinematics
T = eye(4)
for params in dh_params:
    transform = simplify(dh_transform(*params))
    pprint(transform)
    T *= transform
    simplify(T)

pprint(T)

⎡cos(θ₁)  0  sin(θ₁)   0 ⎤
⎢                        ⎥
⎢sin(θ₁)  0  -cos(θ₁)  0 ⎥
⎢                        ⎥
⎢   0     1     0      h₁⎥
⎢                        ⎥
⎣   0     0     0      1 ⎦
⎡-sin(θ₂)  -cos(θ₂)  0  -l₁⋅sin(θ₂)⎤
⎢                                  ⎥
⎢cos(θ₂)   -sin(θ₂)  0  l₁⋅cos(θ₂) ⎥
⎢                                  ⎥
⎢   0         0      1       0     ⎥
⎢                                  ⎥
⎣   0         0      0       1     ⎦
⎡-sin(θ₃)  0  cos(θ₃)  -h₂⋅sin(θ₃)⎤
⎢                                 ⎥
⎢cos(θ₃)   0  sin(θ₃)  h₂⋅cos(θ₃) ⎥
⎢                                 ⎥
⎢   0      1     0          0     ⎥
⎢                                 ⎥
⎣   0      0     0          1     ⎦
⎡cos(θ₄)  0   -sin(θ₄)     0   ⎤
⎢                              ⎥
⎢sin(θ₄)  0   cos(θ₄)      0   ⎥
⎢                              ⎥
⎢   0     -1     0      h₃ + l₂⎥
⎢                              ⎥
⎣   0     0      0         1   ⎦
⎡cos(θ₅)  0  sin(θ₅)   0 ⎤
⎢                        ⎥
⎢sin(θ₅)  0  -c

In [12]:
# Extract end-effector position and orientation
position = T[:3, 3]
R = T[:3, :3]



euler_angles = rotation_to_euler(R)

# Compute the Jacobian
pose = simplify(Matrix.vstack(position, euler_angles))

pprint(pose)

⎡                                  -h₂⋅cos(θ₁)⋅cos(θ₂ + θ₃) - h₃⋅sin(θ₂ + θ₃)⋅
⎢                                                                             
⎢                                  -h₂⋅sin(θ₁)⋅cos(θ₂ + θ₃) - h₃⋅sin(θ₁)⋅sin(θ
⎢                                                                             
⎢                                                                             
⎢                                                                             
⎢atan2((-(sin(θ₁)⋅cos(θ₄)⋅cos(θ₂ + θ₃) + sin(θ₄)⋅cos(θ₁))⋅cos(θ₅) + sin(θ₁)⋅si
⎢                                                                             
⎢                                     ⎛                                       
⎢                                     ⎜                                       
⎢                                atan2⎝(sin(θ₅)⋅cos(θ₂ + θ₃) + sin(θ₂ + θ₃)⋅co
⎢                                                                             
⎣                                                   

In [ ]:
J = simplify(pose.jacobian(theta))

# pprint(pose)

with open("jacobian2.tex", "w") as f:
    f.write(latex(J))

# Partial IK Decomposition (3DOF and Spherical Wrist)

In [22]:
from sympy import *
import numpy as np
# init_printing(use_unicode=True)
init_printing()

In [23]:
def dh_transform(a, alpha, d, theta):
    return Matrix([
        [cos(theta), -sin(theta)*cos(alpha), sin(theta)*sin(alpha), a*cos(theta)],
        [sin(theta), cos(theta)*cos(alpha), -cos(theta)*sin(alpha), a*sin(theta)],
        [0, sin(alpha), cos(alpha), d],
        [0, 0, 0, 1]
    ])

def rotation_to_euler(R):
    yaw = atan2(R[1, 0], R[0, 0])
    pitch = atan2(-R[2, 0], sqrt(R[2, 1]**2 + R[2, 2]**2))
    roll = atan2(R[2, 1], R[2, 2])
    return Matrix([yaw, pitch, roll])

In [ ]:
# Define symbolic joint angles
theta = symbols('theta1:7')

"""
l1: length of first arm
l2: length of second arm
h1: height from base of shoulder motor
h2: offset of second arm (it is L shaped)
h3: offset of spherical wrist center
h4: height of motor (dist from center of spherical wrist to end effector)
"""
l1, l2, h1, h2, h3, h4 = symbols('l1, l2, h1, h2, h3, h4')

dh_params = [
    [0, pi/2, h1, theta[0]], # Joint 1 (base rotation)
    [l1, 0, 0, theta[1] + pi/2], # Joint 2 (shoulder)
    [h2, pi/2, 0, -theta[2] + pi/2], # Joint 3 (elbow)
    [0, -pi/2, l2 + h3, theta[3]], # Joint 4 (wrist 1)
    [0, -pi/2, 0, theta[4]], # Joint 5 (wrist 2)
    [0, 0, h4, theta[5]] # Joint 6 (wrist 3)
]

# Compute forward kinematics
T = eye(4)
for params in dh_params[:4]: # use up to joint 4 for wrist location
    transform = simplify(dh_transform(*params))
    pprint(transform)
    T *= transform
    simplify(T)

pprint(T)

⎡cos(θ₁)  0  sin(θ₁)   0 ⎤
⎢                        ⎥
⎢sin(θ₁)  0  -cos(θ₁)  0 ⎥
⎢                        ⎥
⎢   0     1     0      h₁⎥
⎢                        ⎥
⎣   0     0     0      1 ⎦
⎡-sin(θ₂)  -cos(θ₂)  0  -l₁⋅sin(θ₂)⎤
⎢                                  ⎥
⎢cos(θ₂)   -sin(θ₂)  0  l₁⋅cos(θ₂) ⎥
⎢                                  ⎥
⎢   0         0      1       0     ⎥
⎢                                  ⎥
⎣   0         0      0       1     ⎦
⎡sin(θ₃)  0  cos(θ₃)   h₂⋅sin(θ₃)⎤
⎢                                ⎥
⎢cos(θ₃)  0  -sin(θ₃)  h₂⋅cos(θ₃)⎥
⎢                                ⎥
⎢   0     1     0          0     ⎥
⎢                                ⎥
⎣   0     0     0          1     ⎦
⎡cos(θ₄)  0   -sin(θ₄)     0   ⎤
⎢                              ⎥
⎢sin(θ₄)  0   cos(θ₄)      0   ⎥
⎢                              ⎥
⎢   0     -1     0      h₃ + l₂⎥
⎢                              ⎥
⎣   0     0      0         1   ⎦
⎡(-sin(θ₂)⋅sin(θ₃)⋅cos(θ₁) - cos(θ₁)⋅cos(θ₂)⋅cos(θ₃))⋅cos(θ₄) + sin(θ₁)⋅sin(

In [25]:
position = simplify(T[:3, 3])

pprint(position)

⎡-(h₂⋅cos(θ₂ - θ₃) + h₃⋅sin(θ₂ - θ₃) + l₁⋅sin(θ₂) + l₂⋅sin(θ₂ - θ₃))⋅cos(θ₁)⎤
⎢                                                                           ⎥
⎢-(h₂⋅cos(θ₂ - θ₃) + h₃⋅sin(θ₂ - θ₃) + l₁⋅sin(θ₂) + l₂⋅sin(θ₂ - θ₃))⋅sin(θ₁)⎥
⎢                                                                           ⎥
⎣   h₁ - h₂⋅sin(θ₂ - θ₃) + h₃⋅cos(θ₂ - θ₃) + l₁⋅cos(θ₂) + l₂⋅cos(θ₂ - θ₃)   ⎦


In [ ]:
J = simplify(position.jacobian(theta))

with open("jacobian_3dof.tex", "w") as f:
    f.write(latex(J))

In [21]:
for i in range(3):
    for j in range (3):
        print(f"J{i+1}{j+1} = ", end='')
        pprint(J[i, j])

J11 = (h₂⋅cos(θ₂ - θ₃) + h₃⋅sin(θ₂ - θ₃) + l₁⋅sin(θ₂) + l₂⋅sin(θ₂ - θ₃))⋅sin(θ₁)
J12 = (h₂⋅sin(θ₂ - θ₃) - h₃⋅cos(θ₂ - θ₃) - l₁⋅cos(θ₂) - l₂⋅cos(θ₂ - θ₃))⋅cos(θ₁)
J13 = (-h₂⋅sin(θ₂ - θ₃) + h₃⋅cos(θ₂ - θ₃) + l₂⋅cos(θ₂ - θ₃))⋅cos(θ₁)
J21 = -(h₂⋅cos(θ₂ - θ₃) + h₃⋅sin(θ₂ - θ₃) + l₁⋅sin(θ₂) + l₂⋅sin(θ₂ - θ₃))⋅cos(θ₁)
J22 = (h₂⋅sin(θ₂ - θ₃) - h₃⋅cos(θ₂ - θ₃) - l₁⋅cos(θ₂) - l₂⋅cos(θ₂ - θ₃))⋅sin(θ₁)
J23 = (-h₂⋅sin(θ₂ - θ₃) + h₃⋅cos(θ₂ - θ₃) + l₂⋅cos(θ₂ - θ₃))⋅sin(θ₁)
J31 = 0
J32 = -h₂⋅cos(θ₂ - θ₃) - h₃⋅sin(θ₂ - θ₃) - l₁⋅sin(θ₂) - l₂⋅sin(θ₂ - θ₃)
J33 = h₂⋅cos(θ₂ - θ₃) + h₃⋅sin(θ₂ - θ₃) + l₂⋅sin(θ₂ - θ₃)
